In [2]:
#Basic Functionality 
import pandas as pd
import numpy as np
import math
import sys 
import itertools
import json
import os

#Plot
import matplotlib.pyplot as plt
import seaborn as sns


#Modeling
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

from joblib import dump
import dvc.api
import yaml
import boto3




In [3]:
import tensorflow as tf

print('Tensorflow has access to the following devices:')
for device in tf.config.list_physical_devices():
    print(f'. {device}')




2023-06-28 17:01:59.332039: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Tensorflow has access to the following devices:
. PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
. PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [4]:
! dvc init -f 

Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [5]:
! dvc config core.analytics false

In [6]:
! dvc remote add -d remote_storage /Users/ezrakim/Downloads/dvc_remote

Setting 'remote_storage' as a default remote.


In [7]:
! dvc add data/raw/athletes.csv

                                                                ⠋ Checking graph
Adding...                                                                       
!
  0% Checking cache in '/Users/ezrakim/Documents/GitHub/dvc-test/.dvc/cache/file
                                                                                
!
  0%|          |Checking out data/raw/athletes.csv    0/? [00:00<?,    ?files/s]
  0%|          |Checking out data/raw/athletes.csv    0/1 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00,  7.02file/s]

To track the changes with git, run:

	git add data/raw/athletes.csv.dvc

To enable auto staging, run:

	dvc config core.autostage true


In [8]:
# change number one

raw = pd.read_csv('data/raw/athletes.csv')

raw.head()

,athlete_id,name,region,team,affiliate,gender,age,height,weight,fran,...,snatch,deadlift,backsq,pullups,eat,train,background,experience,schedule,howlong
0,2554.0,Pj Ablang,South West,Double Edge,Double Edge CrossFit,Male,24.0,70.0,166.0,NaN,...,NaN,400.0,305.0,NaN,NaN,I workout mostly at a CrossFit Affiliate|I hav...,I played youth or high school level sports|I r...,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 2x a week|,4+ years|
1,3517.0,Derek Abdella,NaN,NaN,NaN,Male,42.0,70.0,190.0,NaN,...,NaN,NaN,NaN,NaN,NaN,I have a coach who determines my programming|I...,I played youth or high school level sports|,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 2x a week|,4+ years|
2,4691.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5164.0,Abo Brandon,Southern California,LAX CrossFit,LAX CrossFit,Male,40.0,67.0,NaN,211.0,...,200.0,375.0,325.0,25.0,I eat 1-3 full cheat meals per week|,I workout mostly at a CrossFit Affiliate|I hav...,I played youth or high school level sports|,I began CrossFit by trying it alone (without a...,I usually only do 1 workout a day|,4+ years|
4,5286.0,Bryce Abbey,NaN,NaN,NaN,Male,32.0,65.0,149.0,206.0,...,150.0,NaN,325.0,50.0,I eat quality foods but don't measure the amount|,I workout mostly at a CrossFit Affiliate|I inc...,I played college sports|,I began CrossFit by trying it alone (without a...,I usually only do 1 workout a day|I strictly s...,1-2 years|


In [9]:
# drop some columns

v1 = raw.drop(columns=['name', 'schedule'])

v1.head()

,athlete_id,region,team,affiliate,gender,age,height,weight,fran,helen,...,candj,snatch,deadlift,backsq,pullups,eat,train,background,experience,howlong
0,2554.0,South West,Double Edge,Double Edge CrossFit,Male,24.0,70.0,166.0,NaN,NaN,...,220.0,NaN,400.0,305.0,NaN,NaN,I workout mostly at a CrossFit Affiliate|I hav...,I played youth or high school level sports|I r...,I began CrossFit with a coach (e.g. at an affi...,4+ years|
1,3517.0,NaN,NaN,NaN,Male,42.0,70.0,190.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,I have a coach who determines my programming|I...,I played youth or high school level sports|,I began CrossFit with a coach (e.g. at an affi...,4+ years|
2,4691.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5164.0,Southern California,LAX CrossFit,LAX CrossFit,Male,40.0,67.0,NaN,211.0,645.0,...,245.0,200.0,375.0,325.0,25.0,I eat 1-3 full cheat meals per week|,I workout mostly at a CrossFit Affiliate|I hav...,I played youth or high school level sports|,I began CrossFit by trying it alone (without a...,4+ years|
4,5286.0,NaN,NaN,NaN,Male,32.0,65.0,149.0,206.0,465.0,...,205.0,150.0,NaN,325.0,50.0,I eat quality foods but don't measure the amount|,I workout mostly at a CrossFit Affiliate|I inc...,I played college sports|,I began CrossFit by trying it alone (without a...,1-2 years|


In [11]:
v1.to_csv('data/prepare/atheletes_1.csv')

In [12]:
# add to dvc

! dvc add data/prepare/atheletes_1.csv

                                                                ⠋ Checking graph
Adding...                                                                       
!
  0% Checking cache in '/Users/ezrakim/Documents/GitHub/dvc-test/.dvc/cache/file
                                                                                
!
  0%|          |Checking out data/prepare/atheletes_1.0/? [00:00<?,    ?files/s]
  0%|          |Checking out data/prepare/atheletes_1.0/1 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00,  7.82file/s]

To track the changes with git, run:

	git add data/prepare/atheletes_1.csv.dvc data/prepare/.gitignore

To enable auto staging, run:

	dvc config core.autostage true


In [13]:
! dvc push data/prepare/atheletes_1.csv

1 file pushed                                                                   


In [14]:
# make second version

v2 = v1.drop('eat', axis=1)

v2.head()

,athlete_id,region,team,affiliate,gender,age,height,weight,fran,helen,...,run5k,candj,snatch,deadlift,backsq,pullups,train,background,experience,howlong
0,2554.0,South West,Double Edge,Double Edge CrossFit,Male,24.0,70.0,166.0,NaN,NaN,...,NaN,220.0,NaN,400.0,305.0,NaN,I workout mostly at a CrossFit Affiliate|I hav...,I played youth or high school level sports|I r...,I began CrossFit with a coach (e.g. at an affi...,4+ years|
1,3517.0,NaN,NaN,NaN,Male,42.0,70.0,190.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,I have a coach who determines my programming|I...,I played youth or high school level sports|,I began CrossFit with a coach (e.g. at an affi...,4+ years|
2,4691.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5164.0,Southern California,LAX CrossFit,LAX CrossFit,Male,40.0,67.0,NaN,211.0,645.0,...,NaN,245.0,200.0,375.0,325.0,25.0,I workout mostly at a CrossFit Affiliate|I hav...,I played youth or high school level sports|,I began CrossFit by trying it alone (without a...,4+ years|
4,5286.0,NaN,NaN,NaN,Male,32.0,65.0,149.0,206.0,465.0,...,1081.0,205.0,150.0,NaN,325.0,50.0,I workout mostly at a CrossFit Affiliate|I inc...,I played college sports|,I began CrossFit by trying it alone (without a...,1-2 years|


In [15]:
v2.to_csv('data/prepare/athletes_2.csv')

In [16]:
! dvc add data/prepare/athletes_2.csv

                                                                ⠋ Checking graph
Adding...                                                                       
!
  0% Checking cache in '/Users/ezrakim/Documents/GitHub/dvc-test/.dvc/cache/file
                                                                                
!
  0%|          |Checking out data/prepare/athletes_2.c0/? [00:00<?,    ?files/s]
  0%|          |Checking out data/prepare/athletes_2.c0/1 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00,  8.80file/s]

To track the changes with git, run:

	git add data/prepare/athletes_2.csv.dvc data/prepare/.gitignore

To enable auto staging, run:

	dvc config core.autostage true


In [17]:
! dvc push data/prepare/athletes_2.csv

1 file pushed                                                                   


In [21]:
! dvc pull /Users/ezrakim/Downloads/dvc_remote/atheletes_1.csv.dvc

ERROR                                                                 : failed to pull data from the cloud - '/Users/ezrakim/Downloads/dvc_remote/atheletes_1.csv.dvc' does not exist


In [26]:
! dvc checkout data/prepare/atheletes_1.csv.dvc #brought back data

A                                                                                      data/prepare/atheletes_1.csv
